In [16]:
import tqdm
def read_news_tsv(file_name):
    data = []
    with open(file_name, encoding="utf-8") as fh:
        for line in tqdm.tqdm(fh.readlines()):
            title, author, url, timestamp, _, _, text, text_tokenized = line.split("\t")
            article = {
                "title": title,
                "author": author,
                "timestamp": timestamp,
                "text": text,
                "text_tokenized": text_tokenized
            }
            data.append(article)
        return data

In [24]:
articles = read_news_tsv("icelandic_news_sample_22480.tsv")
print(f"Article count {len(articles)}")

100%|█████████████████████████████████| 22248/22248 [00:00<00:00, 214817.21it/s]

Article count 22248


In [2]:
import pandas as pd

In [8]:
news_df = pd.read_csv("icelandic_news_sample_22480.tsv", sep='\t', header = None)

In [9]:
print(len(news_df))

21518


In [22]:
news_df.tail()

,0,1,2,3,4,5,6,7
21513,Samherjamálið er hrikalegt áfall fyrir allan s...,Ritstjórn Hringbrautar,http://www.hringbraut.is/frettir/samherjamalid...,2019-11-13 14:30:00,2019-11-13 15:00:17.237825,2020-10-20 05:34:40.142362,Ef marka má mikinn fréttaflutning og fram komi...,Ef marka má mikinn fréttaflutning og fram komi...
21514,HeForShe berst gegn netníði: Konur á aldrinum ...,Hulda Hólmkelsdóttir,http://www.visir.is/heforshe-berst-gegn-netnid...,2016-10-11 11:15:00,2016-10-11 12:00:12.706579,2020-10-14 06:23:56.773503,UN Women á Íslandi hleypir í dag af stokkunum ...,UN Women á Íslandi hleypir í dag af stokkunum ...
21515,Hönnuður New York-ástar­lógósins er látinn,Atli Ísleifsson,https://www.visir.is/g/20201985728d/honnudur-n...,2020-06-27 18:17:00,2020-06-27 18:30:13.330922,2020-06-27 18:30:26.505181,"Grafíski hönnuðurinn Milton Glaser, sem hannað...","Grafíski hönnuðurinn Milton Glaser , sem hanna..."
21516,Dagbjartur og Ragnhildur sigruðu Síma-mótið,Anton Ingi Leifsson,https://www.visir.is/g/2019190608898/dagbjartu...,2019-06-09 16:14:00,2019-06-09 16:30:12.727944,2020-08-30 05:21:12.769194,"Dagbjartur Sigurbrandsson, úr GR, og Ragnhildu...","Dagbjartur Sigurbrandsson , úr GR , og Ragnhil..."
21517,Elliði hafnar því að líkja Samfylkingunni við ...,Urður Örlygsdóttir,https://www.frettabladid.is/frettir/ellidi-haf...,2020-05-05 23:12:00,2020-05-05 23:30:12.774491,2020-05-31 01:58:02.246298,Í samtali við Fréttablaðið segir Elliði Vignis...,Í samtali við Fréttablaðið segir Elliði Vignis...


In [27]:
articles[-10:]

[{'title': 'Óttast að endanleg tala látinna verði sláandi',
  'author': 'Þórgnýr Einar Albertsson',
  'timestamp': '2019-09-06 19:00:00',
  'text': 'Að minnsta kosti þrjátíu andlát hafa nú verið staðfest á Bahamaeyjum eftir að fellibylurinn Dorian reið þar yfir. Búist er við því að tala látinna haldi áfram að hækka. Duane Sands, heilbrigðismálaráðherra eyjanna, varaði við því í dag að endanleg tala gæti orðið sláandi. Hundraða, jafnvel þúsunda, er enn saknað. Þá er talið að um helmingur heimila á þeim eyjum sem urðu verst úti sé verulega skemmdur eða gjörónýtur. Dorian telst nú fyrsta stigs fellibylur og er mun veikari en þegar hann gekk á land á Bahamaeyjum. Stormurinn hélt áfram að valda hættulegum sjávarflóðum í Norður-og Suður-Karólínu í Bandaríkjunum í dag. Rafmagnslaust hefur verið víða og heimili eyðilagst.',
  'text_tokenized': 'Að minnsta kosti þrjátíu andlát hafa nú verið staðfest á Bahamaeyjum eftir að fellibylurinn Dorian reið þar yfir . Búist er við því að tala látinna hal

In [17]:
articles_Big = read_news_tsv("articles_until_01-11-2020_detok.tsv")
print(f"Article count {len(articles_Big)}")

100%|███████████████████████████████| 640136/640136 [00:05<00:00, 114403.30it/s]


Article count 640136


In [18]:
news_Big = pd.read_csv("articles_until_01-11-2020_detok.tsv", sep='\t', header = None)

In [19]:
print(len(news_Big))

605505


In [20]:
news_Big.shape

(605505, 8)

In [21]:
news_Big.head()

,0,1,2,3,4,5,6,7
0,Hádegisfréttir: Uppsagnir áfall fyrir Reykjanesbæ,Anna Sigríður Einarsdóttir,https://www.ruv.is/frett/2020/08/30/hadegisfre...,2020-08-30 12:22:52,2020-08-30 12:30:11.554767,2020-08-30 12:30:23.379201,Uppsögn 133 starfsmanna ISAVIA á föstudag er á...,Uppsögn 133 starfsmanna ISAVIA á föstudag er á...
1,Áfram aðskilnaður skýlis og stöðvar,Ritstjórn mbl.is,https://mbl.is/frettir/innlent/2020/08/17/afra...,2020-08-17 08:18:00,2020-08-17 08:30:13.072843,2020-08-17 08:30:30.324871,Strætisvagnabiðstöð á hringtorgi við Hádegismó...,Strætisvagnabiðstöð á hringtorgi við Hádegismó...
2,Styrkveitingar til framboðs Eyþórs mistök eða ...,Bára Huld Beck,https://kjarninn.is/frettir/2018-05-11-styrkve...,2018-05-11 13:04:27,2018-05-11 13:30:15.276715,2020-10-06 05:42:13.545689,"Egill Sigurðsson, stjórnarformaður Mjólkursams...","Egill Sigurðsson , stjórnarformaður Mjólkursam..."
3,Ívar: Hefði verið rekinn ef Snæfells-leikurinn...,Henry Birgir Gunnarsson,http://www.visir.is/ivar--hefdi-verid-rekinn-e...,2017-03-05 22:23:43,2017-03-05 22:30:11.243882,2020-10-10 05:27:41.2636,"Það var þungu fargi létt af Ívari Ásgrímssyni,...",Það var þungu fargi létt af Ívari Ásgrímssyni ...
4,„Þetta er ekki fyrsta brottvísunin“,Ritstjórn mbl.is,https://mbl.is/frettir/innlent/2020/09/15/er_e...,2020-09-15 13:58:00,2020-09-15 14:30:18.144264,2020-09-15 14:30:33.558023,"„Þetta er ekki fyrsta brottvísunin þetta mál,“...",„ Þetta er ekki fyrsta brottvísunin þetta mál ...
